In [2]:
import torch
from torch import nn 
from torch.nn import functional as F

In [12]:
X=torch.rand(5,20)

net=nn.Sequential(nn.Linear(X.shape[-1],10)
                , nn.ReLU()
                ,nn.Linear(10,5))
net(X)

tensor([[-0.0026,  0.1767, -0.3312, -0.1216,  0.5105],
        [-0.1204,  0.0231, -0.2551, -0.0735,  0.3413],
        [ 0.0213,  0.1025, -0.1686, -0.0715,  0.4109],
        [-0.1418,  0.1543, -0.1358, -0.1538,  0.4805],
        [-0.0527,  0.0540, -0.2521, -0.2497,  0.5389]],
       grad_fn=<AddmmBackward>)

In [20]:

net[2].state_dict()['weight']

tensor([[ 0.3139, -0.3123,  0.1254,  0.2891,  0.0539,  0.1641, -0.0206,  0.0501,
          0.3057,  0.2780],
        [ 0.2714, -0.0846, -0.0441,  0.2475, -0.1414, -0.1834,  0.2441, -0.0888,
         -0.0567,  0.2860],
        [-0.2193,  0.0879,  0.1632,  0.2616, -0.2979, -0.0314,  0.3156,  0.2557,
         -0.0577, -0.1646],
        [-0.1920, -0.1722,  0.1938,  0.2071, -0.0776, -0.2483, -0.2119,  0.1683,
          0.1146, -0.0840],
        [ 0.2013,  0.2090,  0.1560,  0.0972, -0.0391,  0.1905,  0.3033, -0.1147,
         -0.1207,  0.2506]])

In [22]:
#because we havenot invoked backpropagation 
# for this network yet, it is in its initial state
net[0].weight.grad==None

True

In [31]:
for i, j in net.named_parameters():
    print([i,j.shape])

['0.weight', torch.Size([10, 20])]
['0.bias', torch.Size([10])]
['2.weight', torch.Size([5, 10])]
['2.bias', torch.Size([5])]


In [38]:
net.state_dict()['2.weight']

tensor([[ 0.3139, -0.3123,  0.1254,  0.2891,  0.0539,  0.1641, -0.0206,  0.0501,
          0.3057,  0.2780],
        [ 0.2714, -0.0846, -0.0441,  0.2475, -0.1414, -0.1834,  0.2441, -0.0888,
         -0.0567,  0.2860],
        [-0.2193,  0.0879,  0.1632,  0.2616, -0.2979, -0.0314,  0.3156,  0.2557,
         -0.0577, -0.1646],
        [-0.1920, -0.1722,  0.1938,  0.2071, -0.0776, -0.2483, -0.2119,  0.1683,
          0.1146, -0.0840],
        [ 0.2013,  0.2090,  0.1560,  0.0972, -0.0391,  0.1905,  0.3033, -0.1147,
         -0.1207,  0.2506]])

In [3]:
#Nested Block
X=torch.rand(5,20)
def block():
    return nn.Sequential(nn.Linear(X.shape[1],10)
                        , nn.ReLU()
                        ,nn.Linear(10,20)
                        ,nn.ReLU())
def block2():
    net=nn.Sequential()
    num_blocks=5
    for i in range(num_blocks):
        net.add_module(f'block_{i}',block())
    return net
nested_net=block2()

In [4]:
nested_net.add_module('linear',nn.Linear(20,4))

In [5]:
nested_net(X)

tensor([[0.0663, 0.0826, 0.0071, 0.1266],
        [0.0663, 0.0826, 0.0071, 0.1266],
        [0.0663, 0.0826, 0.0071, 0.1266],
        [0.0663, 0.0826, 0.0071, 0.1266],
        [0.0663, 0.0826, 0.0071, 0.1266]], grad_fn=<AddmmBackward>)

Linear(in_features=20, out_features=10, bias=True)

In [8]:
#parameter Intialization
def initialize_normal(layer):
    if type(layer)==nn.Linear:
        nn.init.normal_(layer.weight,mean=0,std=0.01)
        nn.init.zeros_(layer.bias)
nested_net.apply(initialize_normal)

Sequential(
  (block 0): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 1): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 2): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 3): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 4): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (linear): Linear(in_features=20, out_features=4, bias=True)
)

In [9]:
def initialize_constant(layer):
    if type(layer)==nn.Linear:
        nn.init.constant_(layer.weight,1)
        nn.init.zeros_(layer.bias)
nested_net.apply(initialize_constant)

Sequential(
  (block 0): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 1): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 2): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 3): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 4): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (linear): Linear(in_features=20, out_features=4, bias=True)
)

In [13]:
def xavier(layer):
    if type(layer)==nn.Linear:
        nn.init.xavier_uniform_(layer.weight)
    
nested_net.apply(xavier)

Sequential(
  (block 0): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 1): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 2): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 3): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 4): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (linear): Linear(in_features=20, out_features=4, bias=True)
)

In [23]:
nested_net[0][0].weight.data*nested_net[0][0].weight.data.abs()>=5

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, Fa

In [25]:
def custom_initialization(layer):
    if type(layer)==nn.Linear:
        nn.init.uniform_(layer.weight,-10,10)
        layer.weight.data*=layer.weight.data.abs()>=5


nested_net.apply(custom_initialization)

Sequential(
  (block 0): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 1): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 2): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 3): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (block 4): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU()
  )
  (linear): Linear(in_features=20, out_features=4, bias=True)
)